In [ ]:
from api_setup import logger_setup, set_env_variables

logger_setup()
set_env_variables()

In [ ]:
import spotipy #type: ignore
from spotipy.oauth2 import SpotifyClientCredentials, SpotifyOAuth #type: ignore
from typing import Union, Any
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

## Plan

1. Get list of songs in each playlist
2. Get details from all songs

In [ ]:
def get_playlists() -> list[str]:
    """
    Gets playlist codes from secrets and loads into a list
    """

    playlists: list[str] = []

    with open('secrets/playlists.txt','r') as f:
        lines = f.readlines() 

        for line in lines:
            playlist_id = line[line.find('=')+1:-1]
            if playlist_id:
                playlists.append(playlist_id)
    
    return playlists

def get_songs_in_playlist(sp:spotipy.Spotify,pl_id:str)->dict[str,list[dict[str,dict[str,str]]]]:
    """
    Accesses the Spotify API to get the IDs of the songs with a given Playlist ID.
    """
    
    offset = 0

    response = sp.playlist_items(pl_id,
                                offset=offset,
                                fields='items.track.id,total',
                                additional_types=['track'])
    
    return response

def create_total_track_list(sp:spotipy.Spotify) -> tuple[set[str],list[dict[str,Union[str, list[str]]]]]:
    """
    Creates list of all tracks in each playlist
    """

    playlists: list[str] = get_playlists()

    track_ids: set[str] = set()
    playlist_contents: list[dict[str,Union[str, list[str]]]] = []
    
    for playlist in playlists:
        response = get_songs_in_playlist(sp,playlist)
        current_playlist_contents: dict[str,Union[str, list[str]]] = {'playlist_id':playlist, 'song_ids':response['items']}
        playlist_contents.append(current_playlist_contents)

        for item in response['items']:
            track_ids.add(item['track']['id'])

    return track_ids, playlist_contents

    

In [ ]:
def get_song_properties(sp: spotipy.Spotify, track_ids: set[str]):
    track_list: list[str] = list(track_ids)
    n_tracks = len(track_list)
    track_features: list[dict[str,Any]] = []

    analysed_tracks: int = 0
    while analysed_tracks < n_tracks-1:
        response = sp.audio_features(track_list[analysed_tracks:analysed_tracks+99])
        analysed_tracks += 99
        track_features += response

    return track_features

        

In [ ]:
client_credentials_manager = SpotifyClientCredentials()
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
scope = 'playlist-read-collaborative'
sp_personal = spotipy.Spotify(auth_manager=SpotifyOAuth(scope=scope))
tracklist, playlist_contents = create_total_track_list(sp)

In [ ]:
track_features = get_song_properties(sp, tracklist)

In [ ]:
dat_track_features = pd.DataFrame(track_features)
list(dat_track_features)

In [ ]:
graph_track_features = [
    'danceability',
    'energy',
    'loudness',
    'speechiness',
    'acousticness',
    'instrumentalness',
    'liveness',
    'valence',
    'tempo',
    'duration_ms']

categorical_track_features = [
    'key',
    'mode',
    'time_signature'
]

#### Danceability
Danceability describes how suitable a track is for dancing based on a combination of musical elements including tempo, rhythm stability, beat strength, and overall regularity. A value of 0.0 is least danceable and 1.0 is most danceable.
#### Energy
Energy is a measure from 0.0 to 1.0 and represents a perceptual measure of intensity and activity. Typically, energetic tracks feel fast, loud, and noisy. For example, death metal has high energy, while a Bach prelude scores low on the scale. Perceptual features contributing to this attribute include dynamic range, perceived loudness, timbre, onset rate, and general entropy.
#### Key
The key the track is in. Integers map to pitches using standard Pitch Class notation. E.g. 0 = C, 1 = C♯/D♭, 2 = D, and so on. If no key was detected, the value is -1. >= -1 <= 11
#### Loudness
The overall loudness of a track in decibels (dB). Loudness values are averaged across the entire track and are useful for comparing relative loudness of tracks. Loudness is the quality of a sound that is the primary psychological correlate of physical strength (amplitude). Values typically range between -60 and 0 db.
#### Mode
Mode indicates the modality (major or minor) of a track, the type of scale from which its melodic content is derived. Major is represented by 1 and minor is 0.
#### Speechiness
Speechiness detects the presence of spoken words in a track. The more exclusively speech-like the recording (e.g. talk show, audio book, poetry), the closer to 1.0 the attribute value. Values above 0.66 describe tracks that are probably made entirely of spoken words. Values between 0.33 and 0.66 describe tracks that may contain both music and speech, either in sections or layered, including such cases as rap music. Values below 0.33 most likely represent music and other non-speech-like tracks.
#### Acousticness
A confidence measure from 0.0 to 1.0 of whether the track is acoustic. 1.0 represents high confidence the track is acoustic.
#### Instrumentalness
Predicts whether a track contains no vocals. "Ooh" and "aah" sounds are treated as instrumental in this context. Rap or spoken word tracks are clearly "vocal". The closer the instrumentalness value is to 1.0, the greater likelihood the track contains no vocal content. Values above 0.5 are intended to represent instrumental tracks, but confidence is higher as the value approaches 1.0.
#### Liveness
Detects the presence of an audience in the recording. Higher liveness values represent an increased probability that the track was performed live. A value above 0.8 provides strong likelihood that the track is live.
#### Valence
A measure from 0.0 to 1.0 describing the musical positiveness conveyed by a track. Tracks with high valence sound more positive (e.g. happy, cheerful, euphoric), while tracks with low valence sound more negative (e.g. sad, depressed, angry). >= 0 <= 1
#### Tempo
The overall estimated tempo of a track in beats per minute (BPM). In musical terminology, tempo is the speed or pace of a given piece and derives directly from the average beat duration.
#### Type
The type of body. Always "audio features".
#### ID
The Spotify ID for the track.
#### URI
The Spotify URI for the track.
#### Track_href
The URL for the song
#### Analysis_url
A URL to access the full audio analysis of this track. An access token is required to access this data.
#### Duration_ms
The duration of the track in milliseconds.
#### Time_signature
An estimated time signature. The time signature (meter) is a notational convention to specify how many beats are in each bar (or measure). The time signature ranges from 3 to 7 indicating time signatures of "3/4", to "7/4". >= 3 <= 7

In [ ]:
dat_track_features.describe()

In [ ]:
dat_graph = dat_track_features[graph_track_features]

In [ ]:
sns.pairplot(dat_graph)
plt.show()

In [ ]:
# Create list of data frames
for playlist in playlist_contents:
    data = playlist['song_ids']
    playlist.update({'dataframe':pd.DataFrame(list(tracks['track']['id'] for tracks in data),columns=['id'])})
    playlist['dataframe'] = playlist['dataframe'].merge(dat_track_features, on='id', how='left')

In [ ]:
playlist_contents[0]['dataframe'].head()

In [ ]:
for playlist in playlist_contents:
    averages = playlist['dataframe'].mean()
    playlist.update({'danceability':averages['danceability'],
                     'energy':averages['energy'],
                     'loudness':averages['loudness'],
                     'mode':averages['mode'],
                     'speechiness':averages['speechiness'],
                     'acousticness':averages['acousticness'],
                     'instrumentalness':averages['instrumentalness'],
                     'liveness':averages['liveness'],
                     'valence':averages['valence'],
                     'tempo':averages['tempo'],
                     'duration_ms':averages['duration_ms']})

In [ ]:
playlist_contents[0].keys()

In [ ]:
playlist_id = []
danceability = []
energy = []
loudness = []
mode = []
speechiness = []
acousticness = []
instrumentalness = []
liveness = []
valence = []
tempo = []
duration_ms = []

for playlist in playlist_contents:
    playlist_id.append(playlist['playlist_id'])
    danceability.append(playlist['danceability'])
    energy.append(playlist['energy'])
    loudness.append(playlist['loudness'])
    mode.append(playlist['mode'])
    speechiness.append(playlist['speechiness'])
    acousticness.append(playlist['acousticness'])
    instrumentalness.append(playlist['instrumentalness'])
    liveness.append(playlist['liveness'])
    valence.append(playlist['valence'])
    tempo.append(playlist['tempo'])
    duration_ms.append(playlist['duration_ms'])

playlist_averages = pd.DataFrame({'playlist_id':playlist_id,
                                  'danceability':danceability,
                                  'energy':energy,
                                  'loudness':loudness,
                                  'mode':mode,
                                  'speechiness':speechiness,
                                  'acousticness':acousticness,
                                  'instrumentalness':instrumentalness,
                                  'liveness':liveness,
                                  'valence':valence,
                                  'tempo':tempo,
                                  'duration_ms':duration_ms})

In [ ]:
playlist_averages.head()

Parse data and allocate to people

In [ ]:
rand_names = ['Oliver','Declan','Niamh','Aurora','Silas','Luna']

In [ ]:
playlist_owner_data = []
real_names_list = [] 

with open('secrets\playlists.txt') as f:
    for line in f.readlines():
        playlist_id = line[line.find('=')+1:-1]
        name_date = line[0:line.find('=')].split('_')
        name_date.append(playlist_id)
        
        # Obscure real names
        if name_date[0] not in real_names_list:
            real_names_list.append(name_date[0])
        
        name_date[0] = rand_names[real_names_list.index(name_date[0])]
        playlist_owner_data.append(name_date)

dat_playlist_owner_data = pd.DataFrame(playlist_owner_data, columns=['name', 'month', 'year', 'playlist_id'])

In [ ]:
dat_playlist_owner_data.head()

In [ ]:
month_match = pd.DataFrame({'month':['jan','feb','mar','apr','may','jun','jul','aug','sep','oct','nov','dec'],
                            'month_nr':[1,2,3,4,5,6,7,8,9,10,11,12]})

In [ ]:
owner_playlist_averages = dat_playlist_owner_data.merge(playlist_averages,on='playlist_id').merge(month_match,on='month')
owner_playlist_averages = owner_playlist_averages.assign(date_month = lambda x: pd.to_datetime({'year':"20"+x.year, 'month':x.month_nr, 'day':[1]*len(owner_playlist_averages)},format="%y%m%d"))
owner_playlist_averages = owner_playlist_averages.sort_values(by=['date_month'])
owner_playlist_averages.head()


In [ ]:
sns.lineplot(data=owner_playlist_averages, x='date_month', y='energy', hue='name')


In [ ]:
name_list = owner_playlist_averages['name'].unique()
counter = 0

graph_features = ['danceability',
                  'energy',
                  'loudness',
                  'mode',
                  'speechiness',
                  'acousticness',
                  'instrumentalness',
                  'liveness',
                  'valence',
                  'tempo',
                  'duration_ms']

fig, axs = plt.subplots(nrows=4, ncols=3, figsize=(10,10))

for i in range(4):
    for j in range(3):
        if i == 3 and j == 2:
            continue
        for active_name in name_list:
            active_data = owner_playlist_averages.query('name == @active_name')
            axs[i,j].plot(active_data['date_month'], active_data[graph_features[counter]], )
            axs[i,j].title.set_text(graph_features[counter])
            axs[i,j].xaxis.set_tick_params(rotation=30)
            if graph_features[counter] != "loudness" and graph_features[counter] != "tempo" and graph_features[counter] != "duration_ms":
                axs[i,j].set_ylim(0,1)
        
        counter += 1

axs[3,2].axis('off')
fig.legend(name_list,loc='lower right')
fig.suptitle("Average Playlist Feature Scores per Month")
fig.tight_layout()
plt.show()

In [ ]:
dat_playlist_owner_data.head().merge(month_match,on='month')

In [ ]:
playlist_contents[0].keys()

In [ ]:
for year in range(21,24):
    for month in range(1,13):
        print(year," ", month)